In [2]:
import pandas as pd
import requests

In [ ]:
stages = pd.read_csv('stages.csv')
stages.head()

In [286]:
profile_score = []
for i in range(1,22):
    url = f'https://www.procyclingstats.com/race/tour-de-france/2020/stage-{i}/today/profile-score'
    stage_url = requests.get(url)
    body = stage_url.text
    stage_data = pd.read_html(body)
    stage_data= pd.DataFrame(stage_data[0])
    score = stage_data.loc[len(stage_data)-1,'Score']
    profile_score.append([i,score])
pd.DataFrame(profile_score,columns=['Stage no','profile score']).to_csv('profile_score.csv',index=False)

In [385]:
def climb_score(steep,distance,km0,kmfinish):
    location = kmfinish-km0
    if location<=10:
        factor = 1.0
    elif location <=25:
        factor = 0.8
    elif location <= 50:
        factor = 0.6
    elif location <= 75:
        factor = 0.4
    else:
        factor = 0.2
    return (steep/2)**2*distance * factor
#score = 0
score += climb_score(5,2.7,152.5,198)
score

37.215

In [272]:
import re
import unidecode

wr_riders = []

for i in WR.Rider:
    #s = re.sub("@[A-Za-z0-9_]+ ","", i)
    s = unidecode.unidecode(i)
    wr_riders.append(s.lower())

for stage_n in range(1,22):
    #url = f'https://www.letour.fr/en/rankings/stage-{stage_n}'
    url = f'https://www.procyclingstats.com/race/tour-de-france/2020/stage-{stage_n}'
    stage_url = requests.get(url)
    print(stage_n,stage_url)
    body = stage_url.text
    stage_data = pd.read_html(body)
    stage_data = pd.DataFrame(stage_data[0])
    url = f'https://www.procyclingstats.com/race/tour-de-france/2020/stage-{stage_n}/today/complementary-results'
    stage_url = requests.get(url)
    body = stage_url.text
    stage_points = pd.read_html(body)
    stage_data.Rider = stage_data.Rider.str.lower().apply(lambda x: unidecode.unidecode(x))
    stage_data.Rider = remove_team_name(stage_data)
    stage_data['sprint_pts'] = 0
    stage_data['kom_pts'] = 0
    stage_data['stage_no'] = stage_n
    if stage_n == 1:
        data_full = pd.DataFrame(columns=stage_data.columns)
    for i in stage_points:
        table = pd.DataFrame(i)
        try:
            if len(i)==15:
                # Sprint
                for j in range(16):
                    rider = table.loc[j,'Rider']
                    #rider = straight_names([rider.lower()])
                    rider = unidecode.unidecode(rider.lower())
                    pts = table.loc[j,'Points']
                    try:
                        rider_idx = stage_data[stage_data.Rider==rider].index[0]
                        stage_data.loc[rider_idx,'sprint_pts'] += pts
                    except:
                        print(rider)
            else :
                # KOM
                for j in range(len(table)):
                    rider = table.loc[j,'Rider']
                    #rider = straight_names([rider.lower()])
                    rider = unidecode.unidecode(rider.lower())
                    pts = table.loc[j,'Points']
                    try:
                        rider_idx = stage_data[stage_data.Rider==rider].index[0]
                        stage_data.loc[rider_idx,'kom_pts'] += pts
                    except:
                        print(rider)
        except:
            pass
    data_full.append(stage_data,ignore_index=True)
    data_full = pd.concat([data_full,stage_data],axis=0,ignore_index=True)
    stage_data.to_csv(f'stage{stage_n}.csv',index=False)

data_full.to_csv('data_full.csv',index=False)
stage_data.head()

1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>
10 <Response [200]>
11 <Response [200]>
12 <Response [200]>
13 <Response [200]>
14 <Response [200]>
15 <Response [200]>
16 <Response [200]>
17 <Response [200]>
18 <Response [200]>
19 <Response [200]>
20 <Response [200]>
21 <Response [200]>


,Rnk,GC,GC-Time,BIB,Rider,Age,Team,UCI,Pnt,Time,sprint_pts,kom_pts,stage_no
0,1,138,+5:32:33,43,bennett sam,29,Deceuninck - Quick Step,120.0,100.0,2:53:322:53:32,61,0,21
1,2,124,+5:11:03,105,pedersen mads,24,Trek - Segafredo,50.0,70.0,",,0:00",30,0,21
2,3,84,+4:04:15,21,sagan peter,30,BORA - hansgrohe,25.0,50.0,",,0:00",20,0,21
3,4,132,+5:28:28,135,kristoff alexander,33,UAE-Team Emirates,15.0,40.0,",,0:00",18,0,21
4,5,135,+5:30:01,128,viviani elia,31,"Cofidis, Solutions Crédits",5.0,32.0,",,0:00",16,0,21


In [279]:
data_full.groupby('stage_no')['kom_pts'].sum().sort_values(ascending=False)

stage_no
17    231
18    153
15    139
8     139
13     93
2      73
9      69
16     57
4      41
6      37
12     31
20     31
14     17
3      10
7       7
1       6
5       2
21      1
19      1
11      1
10      0
Name: kom_pts, dtype: int64

In [255]:
def remove_team_name(row):
    riders = []
    for i in range(len(row)):
        rider = row.loc[i,'Rider']
        team = unidecode.unidecode(row.loc[i,'Team'].lower())
        riders.append(rider.split(team)[0])
    return riders

In [264]:
climb = pd.read_csv('WR-climb2020.csv')
print(climb.head())
TT = pd.read_csv('WR-TT2020.csv')
print(TT.head())
sprint = pd.read_csv('WR-sprint2020.csv')
print(sprint.head())

Rnk  Prev. Diff.             Rider                        Team  Pnts  \
0    1    5.0    ▲4     Pogačar Tadej           UAE-Team Emirates   956   
1    2    1.0    ▼1     Roglič Primož            Team Jumbo-Visma   819   
2    3   12.0    ▲9        Yates Adam            Mitchelton-Scott   560   
3    4   56.0   ▲52      Porte Richie            Trek - Segafredo   535   
4    5   31.0   ▲26  Martin Guillaume  Cofidis, Solutions Crédits   516   

                            Unnamed: 6  
0  Point breakdownStandings per season  
1  Point breakdownStandings per season  
2  Point breakdownStandings per season  
3  Point breakdownStandings per season  
4  Point breakdownStandings per season  
   Rnk  Prev. Diff.               Rider                     Team  Pnts  \
0    1    4.0    ▲3       Ganna Filippo         INEOS Grenadiers   430   
1    2    6.0    ▲4        Dennis Rohan         INEOS Grenadiers   385   
2    3    7.0    ▲4     Evenepoel Remco  Deceuninck - Quick Step   340   
3    4    

In [174]:
stage_data

,Rank,Rider,Rank.1,Rider No.,Team,Times,Gap,B,P,sprint_pts,kom_pts
0,1,alexander kristoff,1,135,UAE TEAM EMIRATES,03h 46' 23'',-,B : 10'',-,0,0
1,2,mads pedersen,2,105,TREK - SEGAFREDO,03h 46' 23'',-,B : 6'',-,0,0
2,3,cees bol,3,203,TEAM SUNWEB,03h 46' 23'',-,B : 4'',-,0,0
3,4,sam bennett,4,43,DECEUNINCK - QUICK - STEP,03h 46' 23'',-,-,-,0,0
4,5,peter sagan,5,21,BORA - HANSGROHE,03h 46' 23'',-,-,-,0,0
...,...,...,...,...,...,...,...,...,...,...,...
170,171,nikias arndt,171,202,TEAM SUNWEB,03h 57' 27'',+ 00h 11' 04'',-,-,0,0
171,172,pavel sivakov,172,7,INEOS GRENADIERS,03h 59' 27'',+ 00h 13' 04'',-,-,0,0
172,173,steff cras,173,152,LOTTO SOUDAL,03h 59' 27'',+ 00h 13' 04'',-,-,0,0
173,174,kevin ledanois,174,84,TEAM ARKEA - SAMSIC,03h 59' 27'',+ 00h 13' 04'',-,-,0,0


In [107]:
url = f'https://www.procyclingstats.com/race/tour-de-france/2020/stage-{stage_n}/today/complementary-results'
stage_url = requests.get(url)
print(stage_n,stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[3]).head()

1 <Response [200]>


,Rank,Rider,Team,Points
0,1,Kristoff Alexander,UAE-Team Emirates,50
1,2,Pedersen Mads,Trek - Segafredo,30
2,3,Bol Cees,Team Sunweb,20
3,4,Bennett Sam,Deceuninck - Quick Step,18
4,5,Sagan Peter,BORA - hansgrohe,16


In [101]:
stage_data

[     Rank               Rider  Rank.1  Rider No.                       Team  \
 0       1  ALEXANDER KRISTOFF       1        135          UAE TEAM EMIRATES   
 1       2       MADS PEDERSEN       2        105           TREK - SEGAFREDO   
 2       3            CEES BOL       3        203                TEAM SUNWEB   
 3       4         SAM BENNETT       4         43  DECEUNINCK - QUICK - STEP   
 4       5         PETER SAGAN       5         21           BORA - HANSGROHE   
 ..    ...                 ...     ...        ...                        ...   
 170   171        NIKIAS ARNDT     171        202                TEAM SUNWEB   
 171   172       PAVEL SIVAKOV     172          7           INEOS GRENADIERS   
 172   173          STEFF CRAS     173        152               LOTTO SOUDAL   
 173   174      KÉVIN LEDANOIS     174         84        TEAM ARKEA - SAMSIC   
 174   175  RAFAEL VALLS FERRI     175         68          BAHRAIN - MCLAREN   
 
             Times             Gap    

In [8]:
# TourDeFrance2019
url = f'https://www.procyclingstats.com/race/tour-de-france/2019/result/result'
stage_url = requests.get(url)
print(stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[0]).to_csv('tdf2019.csv',index=False)
pd.DataFrame(stage_data[0]).sort_values(by='GC')

<Response [200]>


,Rnk,GC,GC-Time,BIB,Rider,Age,Team,UCI,Pnt,Time
28,29,1,+0:00,2,Bernal EganTeam INEOS,22,Team INEOS,NaN,NaN,",,0:00"
29,30,2,+1:11,1,Thomas GeraintTeam INEOS,33,Team INEOS,NaN,NaN,",,0:00"
35,36,3,+1:31,81,Kruijswijk StevenTeam Jumbo-Visma,32,Team Jumbo-Visma,NaN,NaN,",,0:00"
46,47,4,+1:56,12,Buchmann EmanuelBORA - hansgrohe,26,BORA - hansgrohe,NaN,NaN,",,0:00"
66,67,5,+4:05,21,Alaphilippe JulianDeceuninck - Quick Step,27,Deceuninck - Quick Step,NaN,NaN,",,0:20"
...,...,...,...,...,...,...,...,...,...,...
118,119,151,+4:14:39,183,Dowsett AlexTeam Katusha Alpecin,30,Team Katusha Alpecin,NaN,NaN,",,0:51"
32,33,152,+4:19:33,26,Mørkøv MichaelDeceuninck - Quick Step,34,Deceuninck - Quick Step,NaN,NaN,",,0:00"
15,16,153,+4:29:07,182,Debusschere JensTeam Katusha Alpecin,29,Team Katusha Alpecin,NaN,NaN,",,0:00"
145,146,154,+4:31:43,196,Offredo YoannWanty - Gobert Cycling Team,32,Wanty - Gobert Cycling Team,NaN,NaN,1:531:53


In [11]:
# 2019 WorldChampionships
url = f'https://www.procyclingstats.com/race/world-championship/2019'
stage_url = requests.get(url)
print(stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[0]).to_csv('WC2019.csv',index=False)
pd.DataFrame(stage_data[0]).sort_values(by='Rnk')

<Response [200]>


,Rnk,BIB,Rider,Age,Team,UCI,Pnt,Time
0,1,80,Pedersen MadsDenmark,23,Denmark,600.0,350.0,6:27:286:27:28
9,10,144,Costa RuiPortugal,32,Portugal,100.0,70.0,",,1:10"
10,11,28,Colbrelli SonnyItaly,29,Italy,85.0,65.0,",,1:10"
11,12,75,Fuglsang JakobDenmark,34,Denmark,70.0,60.0,",,1:10"
12,13,136,Štybar ZdeněkCzech Republic,33,Czech Republic,60.0,55.0,",,1:10"
...,...,...,...,...,...,...,...,...
97,DNF,193,Arashiro YukiyaJapan,35,Japan,NaN,NaN,-
91,DNF,12,Gilbert PhilippeBelgium,37,Belgium,NaN,NaN,-
195,DNS,159,Kudus MerhawiEritrea,25,Eritrea,NaN,NaN,-
194,DNS,182,Padun MarkUkraine,23,Ukraine,NaN,NaN,-


In [12]:
# 2019 Milano-Sanremo
url = f'https://www.procyclingstats.com/race/milano-sanremo/2019'
stage_url = requests.get(url)
print(stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[0]).to_csv('MS2019.csv',index=False)
pd.DataFrame(stage_data[0]).sort_values(by='Rnk')

<Response [200]>


,Rnk,BIB,Rider,Age,Team,UCI,Pnt,Time
0,1,81,Alaphilippe JulianDeceuninck - Quick Step,26,Deceuninck - Quick Step,500.0,275.0,6:40:146:40:14
9,10,141,Trentin MatteoMitchelton-Scott,29,Mitchelton-Scott,85.0,50.0,",,0:00"
99,100,137,Van der Sande ToshLotto Soudal,28,Lotto Soudal,NaN,5.0,",,3:11"
100,101,23,Cattaneo MattiaAndroni Giocattoli - Sidermec,28,Androni Giocattoli - Sidermec,NaN,5.0,",,3:11"
101,102,85,Richeze MaximilianoDeceuninck - Quick Step,36,Deceuninck - Quick Step,NaN,5.0,",,3:11"
...,...,...,...,...,...,...,...,...
170,DNF,53,Bodnar MaciejBORA - hansgrohe,34,BORA - hansgrohe,NaN,NaN,-
171,DNF,65,Van Hooydonck NathanCCC Team,23,CCC Team,NaN,NaN,-
172,DNF,227,Pedersen CasperTeam Sunweb,23,Team Sunweb,NaN,NaN,-
173,DNF,162,Bertazzo LiamNeri Sottoli - Selle Italia - KTM,27,Neri Sottoli - Selle Italia - KTM,NaN,NaN,-


In [48]:
# World Rankings
url = f'https://www.procyclingstats.com/rankings.php?id=59874&nation=&team=&page=0&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
stage_url = requests.get(url)
print(stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[0]).to_csv('WR2020.csv',index=False)
pd.DataFrame(stage_data[0]).sort_values(by='Rnk')

<Response [200]>


,Rnk,Prev.,Diff.,Rider,Team,Pnts,Unnamed: 6
0,1,6,▲5,Pogačar Tadej,UAE-Team Emirates,1302,Point breakdownStandings per season
1,2,2,-,Roglič Primož,Team Jumbo-Visma,1281,Point breakdownStandings per season
2,3,19,▲16,Fuglsang Jakob,Astana Pro Team,1194,Point breakdownStandings per season
3,4,11,▲7,Evenepoel Remco,Deceuninck - Quick Step,1098,Point breakdownStandings per season
4,5,4,▼1,Ackermann Pascal,BORA - hansgrohe,1048,Point breakdownStandings per season
...,...,...,...,...,...,...,...
195,196,181,▼15,Quintero Carlos,Terengganu Inc. TSG Cycling Team,167,Point breakdownStandings per season
196,197,117,▼80,Van der Sande Tosh,Lotto Soudal,167,Point breakdownStandings per season
197,198,214,▲16,Lyu Xianjing,Hengxiang Cycling Team,166,Point breakdownStandings per season
198,199,190,▼9,Eg Niklas,Trek - Segafredo,164,Point breakdownStandings per season


In [94]:
# World Rankings - climbers
url = f'https://www.procyclingstats.com/rankings.php?id=60840&nation=&team=&page=0&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
stage_url = requests.get(url)
print(stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[0]).to_csv('WR-climb2020.csv',index=False)
pd.DataFrame(stage_data[0]).sort_values(by='Rnk')

<Response [200]>


,Rnk,Prev.,Diff.,Rider,Team,Pnts,Unnamed: 6
0,1,5.0,▲4,Pogačar Tadej,UAE-Team Emirates,956,Point breakdownStandings per season
1,2,1.0,▼1,Roglič Primož,Team Jumbo-Visma,819,Point breakdownStandings per season
2,3,12.0,▲9,Yates Adam,Mitchelton-Scott,560,Point breakdownStandings per season
3,4,56.0,▲52,Porte Richie,Trek - Segafredo,535,Point breakdownStandings per season
4,5,31.0,▲26,Martin Guillaume,"Cofidis, Solutions Crédits",516,Point breakdownStandings per season
...,...,...,...,...,...,...,...
195,196,389.0,▲193,Doubey Fabien,Circus - Wanty Gobert,30,Point breakdownStandings per season
196,197,398.0,▲201,Navarro Daniel,Israel Start-Up Nation,30,Point breakdownStandings per season
197,198,643.0,▲445,Vanhoucke Harm,Lotto Soudal,30,Point breakdownStandings per season
198,199,541.0,▲342,Roosen Timo,Team Jumbo-Visma,29,Point breakdownStandings per season


In [93]:
# World Rankings - sprinters
url = f'https://www.procyclingstats.com/rankings.php?id=60805&nation=&team=&page=0&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
stage_url = requests.get(url)
print(stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[0]).to_csv('WR-sprint2020.csv',index=False)
pd.DataFrame(stage_data[0]).sort_values(by='Rnk')

<Response [200]>


,Rnk,Prev.,Diff.,Rider,Team,Pnts,Unnamed: 6
0,1,3.0,▲2,Ackermann Pascal,BORA - hansgrohe,967.0,Point breakdownStandings per season
1,2,4.0,▲2,Bennett Sam,Deceuninck - Quick Step,736.5,Point breakdownStandings per season
2,3,14.0,▲11,Gaviria Fernando,UAE-Team Emirates,633.5,Point breakdownStandings per season
3,4,18.0,▲14,Démare Arnaud,Groupama - FDJ,604.5,Point breakdownStandings per season
4,5,23.0,▲18,Nizzolo Giacomo,NTT Pro Cycling,543.5,Point breakdownStandings per season
...,...,...,...,...,...,...,...
195,196,136.0,▼60,Gómez Bryan,Equipo Continental Supergiros,71.0,Point breakdownStandings per season
196,197,NaN,NaN,Quintero Carlos,Terengganu Inc. TSG Cycling Team,71.0,Point breakdownStandings per season
197,198,216.0,▲18,Naesen Lawrence,AG2R La Mondiale,71.0,Point breakdownStandings per season
198,199,49.0,▼150,Noppe Christophe,Team Arkéa Samsic,71.0,Point breakdownStandings per season


In [95]:
# World Rankings - sprinters
url = f'https://www.procyclingstats.com/rankings.php?id=60806&nation=&team=&page=0&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
stage_url = requests.get(url)
print(stage_url)
body = stage_url.text
stage_data = pd.read_html(body)
pd.DataFrame(stage_data[0]).to_csv('WR-TT2020.csv',index=False)
pd.DataFrame(stage_data[0]).sort_values(by='Rnk')

<Response [200]>


,Rnk,Prev.,Diff.,Rider,Team,Pnts,Unnamed: 6
0,1,4.0,▲3,Ganna Filippo,INEOS Grenadiers,430,Point breakdownStandings per season
1,2,6.0,▲4,Dennis Rohan,INEOS Grenadiers,385,Point breakdownStandings per season
2,3,7.0,▲4,Evenepoel Remco,Deceuninck - Quick Step,340,Point breakdownStandings per season
3,4,2.0,▼2,Campenaerts Victor,NTT Pro Cycling,281,Point breakdownStandings per season
4,5,3.0,▼2,Küng Stefan,Groupama - FDJ,256,Point breakdownStandings per season
...,...,...,...,...,...,...,...
195,196,108.0,▼88,Jurado Cristopher Robin,Terengganu Inc. TSG Cycling Team,26,Point breakdownStandings per season
196,197,72.0,▼125,Mullen Ryan,Trek - Segafredo,26,Point breakdownStandings per season
197,198,622.0,▲424,Finkšt Tilen,Ljubljana Gusto Santic,26,Point breakdownStandings per season
198,199,580.0,▲381,Turek Daniel,Israel Cycling Academy,26,Point breakdownStandings per season


In [96]:
tdf = pd.read_csv('stage18.csv')

In [97]:
tdf.head()

,Rank,Rider,Rank.1,Rider No.,Team,Times,Gap,B,P
0,1,MICHAL KWIATKOWSKI,1,5,INEOS GRENADIERS,04h 47' 33'',-,B : 15'',-
1,2,RICHARD CARAPAZ,2,3,INEOS GRENADIERS,04h 47' 33'',-,B : 14'',-
2,3,WOUT VAN AERT,3,18,TEAM JUMBO - VISMA,04h 49' 24'',+ 00h 01' 51'',B : 4'',P : 10''
3,4,PRIMOŽ ROGLIC,4,11,TEAM JUMBO - VISMA,04h 49' 26'',+ 00h 01' 53'',-,-
4,5,TADEJ POGACAR,5,131,UAE TEAM EMIRATES,04h 49' 26'',+ 00h 01' 53'',-,-


In [29]:
import numpy as np
np.argmax(tdf.Rider.str.find('TADEJ'))

24

In [ ]:
def lower_case(text):
    return text.lower()

In [31]:
tdf.loc[24]

Rank                        25
Rider            TADEJ POGACAR
Rank.1                      25
Rider No.                  131
Team         UAE TEAM EMIRATES
Times             03h 35' 22''
Gap                          -
B                            -
P                            -
Name: 24, dtype: object

In [49]:
WR = pd.DataFrame(stage_data[0])
WR.tail()

,Rnk,Prev.,Diff.,Rider,Team,Pnts,Unnamed: 6
195,196,181,▼15,Quintero Carlos,Terengganu Inc. TSG Cycling Team,167,Point breakdownStandings per season
196,197,117,▼80,Van der Sande Tosh,Lotto Soudal,167,Point breakdownStandings per season
197,198,214,▲16,Lyu Xianjing,Hengxiang Cycling Team,166,Point breakdownStandings per season
198,199,190,▼9,Eg Niklas,Trek - Segafredo,164,Point breakdownStandings per season
199,200,230,▲30,Flórez Miguel Eduardo,Androni Giocattoli - Sidermec,163,Point breakdownStandings per season


In [50]:
WR.Rider

0              Pogačar Tadej
1              Roglič Primož
2             Fuglsang Jakob
3            Evenepoel Remco
4           Ackermann Pascal
               ...          
195          Quintero Carlos
196       Van der Sande Tosh
197             Lyu Xianjing
198                Eg Niklas
199    Flórez Miguel Eduardo
Name: Rider, Length: 200, dtype: object

In [51]:
import re
import unidecode

wr_riders = []
for i in WR.Rider:
    #s = re.sub("@[A-Za-z0-9_]+ ","", i)
    s = unidecode.unidecode(i)
    wr_riders.append(s.lower())
len(wr_riders)

200

In [53]:
wr_riders.isin(tdf.Rider.str.lower().to_list())

AttributeError: 'list' object has no attribute 'isin'

In [54]:
list1_as_set = set(wr_riders)

intersection = list1_as_set.intersection(tdf.Rider.str.lower().to_list())
print(intersection)

set()


In [56]:
wr_riders[0] in tdf.Rider.str.lower().to_list()

False

In [78]:
def straight_names(names_list):
    ordered_names = []
    for name in names_list:
        if len(name.split())==2:
            ordered_names.append(name.split()[1]+' '+name.split()[0])
        elif len(name.split())==3:
            name_c = name.split()[-1]+' '+name.split()[0]+' '+name.split()[1]
            if name in ['martinez daniel felipe','lopez miguel angel','hagen carl fredrik','sanchez luis leon','jansen amund grondahl','florez miguel eduardo']:
                name_c = name.split()[-2]+' '+name.split()[-1]+' '+name.split()[0]
            ordered_names.append(name_c)
        elif len(name.split())==4:
            name_c = name.split()[-2]+' '+name.split()[-1]+' '+name.split()[0]+' '+name.split()[1]
            if name in['de la cruz david','de la parte victor','van der poel mathieu','van der sande tosh']:
                name_c = name.split()[-1]+' '+name.split()[0]+' '+name.split()[1]+' '+name.split()[2]
            ordered_names.append(name_c)
        else:
            print(name)
    return ordered_names
wr_riders1 = straight_names(wr_riders)

In [85]:
ranked_in_tdf = []
ranked_in_tdf.append([rider for rider in wr_riders1 if rider in tdf.Rider.str.lower().to_list()])
ranked_in_tdf = ranked_in_tdf[0]
len(ranked_in_tdf)

71

In [90]:
tdf[tdf.Rider.str.lower().isin(ranked_in_tdf)]

,Rank,Rider,Rank.1,Rider No.,Team,Times,Gap,B,P
0,1,SAM BENNETT,1,43,DECEUNINCK - QUICK - STEP,03h 35' 22'',-,B : 10'',-
1,2,CALEB EWAN,2,151,LOTTO SOUDAL,03h 35' 22'',-,B : 6'',-
2,3,PETER SAGAN,3,21,BORA - HANSGROHE,03h 35' 22'',-,B : 4'',-
3,4,ELIA VIVIANI,4,128,COFIDIS,03h 35' 22'',-,-,-
4,5,MADS PEDERSEN,5,105,TREK - SEGAFREDO,03h 35' 22'',-,-,-
...,...,...,...,...,...,...,...,...,...
150,151,MIKEL NIEVE,151,168,MITCHELTON - SCOTT,03h 40' 47'',+ 00h 05' 25'',-,-
152,153,NILS POLITT,153,177,ISRAEL START-UP NATION,03h 43' 14'',+ 00h 07' 52'',-,-
154,155,DAVID DE LA CRUZ,155,133,UAE TEAM EMIRATES,03h 45' 30'',+ 00h 10' 08'',-,-
159,160,JULIAN ALAPHILIPPE,160,41,DECEUNINCK - QUICK - STEP,03h 46' 14'',+ 00h 10' 52'',-,-


0       True
1       True
2       True
3       True
4       True
       ...  
159     True
160    False
161    False
162    False
163     True
Name: Rider, Length: 164, dtype: bool